In [ ]:
# import numpy as np
import pandas as pd
import nltk
from nltk.corpus import words

In [2]:
## DATA SETS ---------
df = pd.read_csv("spam.csv", encoding = "latin-1", usecols=["v1", "v2"])

## TRANSFORMS
df.columns = ["Status", "Mensagem"]

print("Linhas x Colunas", df.shape, "\n") 
df.head() 

Linhas x Colunas (5572, 2) 



,Status,Mensagem
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
try:
    nltk.data.find("words")
except LookupError:
    nltk.download("words") 
set_words = set(words.words())    

[nltk_data] Downloading package words to C:\Users\Luis
[nltk_data]     Carlos\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [4]:
df['Mensagem'] = df['Mensagem'].str.replace('\W', ' ') # Limpa a pontuação
df['Mensagem'] = df['Mensagem'].str.lower()  #Transforma tudo em letra pequena

df = df[["Mensagem", "Status"]]

df['Status'] = df['Status'].str.replace('spam', '1') # Limpa o subject
df['Status'] = df['Status'].str.replace('ham', '0') # Limpa o subject

df.head()

,Mensagem,Status
0,go until jurong point crazy available only ...,0
1,ok lar joking wif u oni,0
2,free entry in 2 a wkly comp to win fa cup fina...,1
3,u dun say so early hor u c already then say,0
4,nah i don t think he goes to usf he lives aro...,0


In [12]:
class Perceptron():
    def __init__(self, learningRate, numberIterations): #construtor do perceptron
        self.learningRate = learningRate
        self.numberIterations = numberIterations #AKA EPochs/Epocas

    def classificar(self, data, verify):
        self.listaErros = []

        probs = np.zeros(data.shape[0])

        for i in range(self.numberIterations):
            erros = 0
            mail_count = 0

            for mail, target in zip(data, verify):
                prediction = self.predict(mail)
                update = self.learningRate * (target - prediction)

                if update != 0:
                    weight[1:] += update * mail
                    weight[0] += update

                erros += int(update != 0.0)

                probs[mail_count] = prediction

                mail_count += 1

            self.listaErros.append(erros)

        return probs

    def net_input(self, mail):
        return np.dot(mail, weight[1:]) + weight[0]

    def predict(self, mail):
        return np.where(self.net_input(mail) >= 0.0, 1, -1)


In [ ]:
def generate(ficheiro):                                   # Função generate - Gera as matrizes data e verify
    global data                                           # Variável data definida como global
    data = np.zeros((df.shape[0], len(vocabulary)))        # Criação de uma matriz com número de linhas igual ao número de linhas do ficheiro e com número de colunas igual ao número de palavras (sem repetição) existentes no vocabulário

    global verify                                           # Variável verify definida como global
    verify = np.zeros(df.shape[0])                          # Criação de uma matriz com número de linhas igual ao número de linhas do ficheiro e com uma coluna

    for i in range(df.shape[0]):                          # Ciclo que corre todas as linhas do ficheiro
        email = df.iloc[i, 0].split()                        # ----------------------------------------------------------

        for word in email:                                  # Ciclo que corre todas as palavras de uma mensagem
            if word.lower() in vocabulary:                  # Verificação da existência da palavra no dicionário vocabulary
                data[i, vocabulary[word]] += 1              # Adição de valores à matriz data (+1 no índice da palavra que foi repetida)

                if df.iloc[i, 1] == '1':
                    verify[i] = 1                           # Adição de valores à matriz verify (adiciona 1 para spam)

                elif df.iloc[i, 1] == '0':
                    verify[i] = -1                          # Adição de valores à matriz verify (adiciona -1 para ham)

In [31]:
#Divisão em Sets
train, validate, test = np.split(df.sample(frac=1, random_state=42), [int(.7*len(df)), int(.85*len(df))])   #Split aos 70 para criar o grupo de treino, e aos 85 para criar o grupo de teste e validação


#Resultado dos Splits
print("Dados de Treino")
print(train['Status'].value_counts())
print("Linhas ", len(train.index))

print("")

print("Dados de Teste")
print(test['Status'].value_counts())
print("Linhas ", len(test.index))

print("")

print("Dados de Verificação")
print(validate['Status'].value_counts())
print("Linhas ", len(validate.index))

Dados de Treino
0    3379
1     521
Name: Status, dtype: int64
Linhas  3900

Dados de Teste
0    724
1    112
Name: Status, dtype: int64
Linhas  836

Dados de Verificação
0    722
1    114
Name: Status, dtype: int64
Linhas  836


In [ ]:
#VALIDAÇÃO

In [22]:
def train(ficheiro):                                   # Função train - Recebe um ficheiro e cria um dicionário (vocabulary) das palavras usadas
    for i in range(df.shape[0]):                          # Ciclo que corre todas as mensagens do ficheiro
        email = df.iloc[i, 0].split()                   # Mensagem separada por palavras

        for word in email:                                  # Ciclo que corre todas as palavras de uma mensagem
            if word.lower() not in vocabulary and word.lower() in set_words: # Verificação da existência da palavra no dicionário vocabulary e no set set_words
                vocabulary[word] = len(vocabulary)          # Atribuição de uma posição à palavra analisada no dicionário vocabulary
    generate(str(ficheiro))

In [30]:
if __name__ == "__main__":
    vocabulary = {}
    train(df)
    weight = np.zeros(1 + data.shape[1])
    P = Perceptron(1, 10)
    P.classificar(data, verify)

    generate(df)
    P = Perceptron(0, 1)
    classification = P.classificar(data, verify)
 
    print("Total de Linhas: ", df.shape[0])
    print(df['Status'].value_counts())

    print("Algoritmo")
    print("\nNúmero de mensagens classificadas como spam: ", (sum(classification == 1)))
    print("Número de mensagens classificadas como ham: ", (sum(classification == -1)))
    print("\nPrecisão do algoritmo: ", (sum(classification == verify) / data.shape[0] * 100))


Total de Linhas:  5572
0    4825
1     747
Name: Status, dtype: int64
Algoritmo

Número de mensagens classificadas como spam:  744
Número de mensagens classificadas como ham:  4828

Precisão do algoritmo:  98.63603732950466
